In [69]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf
import sklearn

In [46]:
data_dir = "../data/"
pickle_file = 'tf_data.pickle'

with open(data_dir + pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = np.matrix(save['train_data'], dtype='float32')
    train_labels = np.matrix(save['train_labels'], dtype='float32')
    valid_dataset = np.matrix(save['validate_data'], dtype='float32')
    valid_labels = np.matrix(save['validate_labels'], dtype='float32')
    test_dataset = valid_dataset
    test_labels = valid_labels
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape

Training set (1600, 1024) (1600, 9)
Validation set (400, 1024) (400, 9)


In [123]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

#set to -1 to use all data
train_subset = 1600
input_size = 1024
labels_size = 9

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([input_size, labels_size]))
    #biases = tf.Variable(tf.zeros([labels_size]))
    biases = tf.Variable(tf.zeros([labels_size]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        #This uses sigmoid cross entropy, which allows for multiple labels
        tf.nn.sigmoid_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)

In [ ]:
num_steps = 1000

def f1_score(y_true, y_pred):
    l, w = y_pred.shape
    f1 = np.zeros(l, dtype='float32')
    #print f1.shape
    y_pred = np.array(np.array(np.round(y_pred), dtype=bool), dtype='float32')
    print y_pred[:10]
    #y_true.reshape(9,len(y_pred))
    #print y_pred.shape
    #print y_true.shape
    for i in range(len(y_pred)):
        #print y_pred[i]
        #print np.ravel(y_true[i])
        f1[i]= sklearn.metrics.f1_score(np.ravel(y_true[i]), y_pred[i], average='binary')
        
        #if f1[i] < 1.:
            #print y_true[i]
            #print y_pred[i]
    f1_mean = np.mean(f1)
    return f1_mean 

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print 'Initialized'
    for step in xrange(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 1000 == 0):
            #print predictions.shape
            print 'Loss at step', step, ':', l
            #print "F1 Score:", (valid_labels)
            print 'Training accuracy: ',f1_score(
               train_labels[:train_subset, :], predictions)
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print 'Validation accuracy:', f1_score(
                 valid_labels, valid_prediction.eval())

 Initialized
Loss at step 0 : 62.9439
Training accuracy:  [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]]
0.370552
Validation accuracy: [[ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]]
0.284732
Loss at step 1000 : 7.66478
Training accuracy:  [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. 

In [ ]:
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_labels = save['test_labels']
del save  # hint to help gc free up memory
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape